***Import Required Libraries***

In [0]:
import numpy as np
import struct
from sklearn.decomposition import PCA
from keras.datasets import mnist

Using TensorFlow backend.


***Load data***

Although we can find MNIST dataset from Yann LeCun's official site, I chose a more convenient way to find the dataset from Keras.  
Also, from the code below, we can show that the MNIST database contains 60,000 training and 10,000 testing images, which have $28\times28$ pixels with only greyscale.

In [0]:
(train_data_ori, train_label), (test_data_ori, test_label) = mnist.load_data()
print ("mnist data loaded")
print ("original training data shape:",train_data_ori.shape)
print ("original testing data shape:",test_data_ori.shape)

11493376/11490434 [==============================] - 1s 0us/step
mnist data loaded
original training data shape: (60000, 28, 28)
original testing data shape: (10000, 28, 28)


For the convenience of training, linearize each image from $28\times28$ into an array of size $1\times784$, so that the training and test datasets are converted into 2-dimensional vectors of size $60000\times784$ and $10000\times784$, respectively.

In [0]:
train_data=train_data_ori.reshape(60000,784)
test_data=test_data_ori.reshape(10000,784)
print ("training data shape after reshape:",train_data.shape)
print ("testing data shape after reshape:",test_data.shape)

training data shape after reshape: (60000, 784)
testing data shape after reshape: (10000, 784)


***Dimension Reduction using PCA***  

For this case, the pixels of the image will be the features used to build our predictive model. In this way, implementing KNN clustering is to calculate the norms in a 784-dimensional space.  
However, calculating norms in this 784-dimensional space is far from easy and efficient. Intuitively, we can perform some dimention reduction before going to KNN and calculate those norms, so that we can become more efficient.  
The way to do dimension reduction here is PCA mentioned in the lecture. I don't dig deep into PCA here, and use APIs from sklearn to implement PCA instead. I reduce the feature space from 784 dimensions into 100 dimensions. Talk is cheap, here's the code.

In [0]:
pca = PCA(n_components = 100)
pca.fit(train_data) #fit PCA with training data instead of the whole dataset
train_data_pca = pca.transform(train_data)
test_data_pca = pca.transform(test_data)
print("PCA completed with 100 components")
print ("training data shape after PCA:",train_data_pca.shape)
print ("testing data shape after PCA:",test_data_pca.shape)

PCA completed with 100 components
training data shape after PCA: (60000, 100)
testing data shape after PCA: (10000, 100)


From the result above, we can know that the training and test datasets become two vectors of size $60000\times100$ and $10000\times100$, respectively.  

At this point, the datasets are ready.

***Code up KNN***  
Here's the code to K Nearest Neighbor clustering algorithm. This function takes in the image to cluster, training dataset, training labels, the value of K and the sort of norm to calculate distance(*i.e.* the value of $p$ in $l_p$ norm).  
Under the most circumstance, we use Euclidean norm to calculate distace, thus $p=2$.  
This function returns the class most common among the test data's K nearest neighbors, where K is the parameter mentioned above.

In [0]:
def KNN(test_data1,train_data_pca,train_label,k,p):
    subMat = train_data_pca - np.tile(test_data1,(60000,1))
    subMat = np.abs(subMat)
    distance = subMat**p
    distance = np.sum(distance,axis=1)
    distance = distance**(1.0/p)
    distanceIndex = np.argsort(distance)
    classCount = [0,0,0,0,0,0,0,0,0,0]
    for i in range(k):
        label = train_label[distanceIndex[i]]
        classCount[label] = classCount[label] + 1    
    return np.argmax(classCount)

***Define the test function***  
This function takes in the value of K and the value of $p$ in $l_p$ norm mentioned above, and returns the accuracy of KNN clustering on the test dataset, along with the confusion matrix.

In [0]:
def test(k,p):
    print("testing with K= %d and lp norm p=%d"%(k,p))
    m,n = np.shape(test_data_pca)
    correctCount = 0
    M = np.zeros((10,10),int)
    for i in range(m):
        test_data1 = test_data_pca[i,:]
        predict_label = KNN(test_data1,train_data_pca,train_label, k, p)
        true_label = test_label[i]
        M[true_label][predict_label] += 1
#        print("predict：%d,true:%d" % (predict_label,true_label))
        if true_label == predict_label:
           correctCount += 1
                  
    print("The accuracy is: %f" % (float(correctCount)/m))
    print("Confusion matrix:",M)

***Test result***  
Here's the precision of the KNN clustering algorithm with argument K=3 and Euclidean norm, along with the confusion matrix.

In [0]:
test(3,2)

testing with K= 3 and lp norm p=2
The accuracy is: 0.973500
Confusion matrix: [[ 974    1    1    0    0    1    2    1    0    0]
 [   0 1131    3    0    0    0    1    0    0    0]
 [   7    4 1004    1    1    0    0   13    2    0]
 [   1    1    4  979    1    9    0    7    5    3]
 [   2    5    0    0  949    0    4    3    0   19]
 [   4    1    0   10    2  865    3    1    2    4]
 [   4    3    0    0    2    4  945    0    0    0]
 [   0   17    6    0    2    0    0  996    0    7]
 [   5    1    4   17    5   10    5    3  921    3]
 [   5    5    2    8    8    2    1    6    1  971]]


From the result above, we can show that we achieved the precision of 0.972900, with dimension reduction (using PCA) and KNN clustering.